## Day 12

### Part 1

Your ferry made decent progress toward the island, but the storm came in faster than anyone expected. The ferry needs to take evasive actions!

Unfortunately, the ship's navigation computer seems to be malfunctioning; rather than giving a route directly to safety, it produced extremely circuitous instructions. When the captain uses the PA system to ask if anyone can help, you quickly volunteer.

The navigation instructions (your puzzle input) consists of a sequence of single-character actions paired with integer input values. After staring at them for a few minutes, you work out what they probably mean:

Action N means to move north by the given value.
Action S means to move south by the given value.
Action E means to move east by the given value.
Action W means to move west by the given value.
Action L means to turn left the given number of degrees.
Action R means to turn right the given number of degrees.
Action F means to move forward by the given value in the direction the ship is currently facing.
The ship starts by facing east. Only the L and R actions change the direction the ship is facing. (That is, if the ship is facing east and the next instruction is N10, the ship would move north 10 units, but would still move east if the following action were F.)

For example:

- F10
- N3
- F7
- R90
- F11

These instructions would be handled as follows:

- F10 would move the ship 10 units east (because the ship starts by facing east) to east 10, north 0.
- N3 would move the ship 3 units north to east 10, north 3.
- F7 would move the ship another 7 units east (because the ship is still facing east) to east 17, north 3.
- R90 would cause the ship to turn right by 90 degrees and face south; it remains at east 17, north 3.
- F11 would move the ship 11 units south to east 17, south 8.

At the end of these instructions, the ship's Manhattan distance (sum of the absolute values of its east/west position and its north/south position) from its starting position is 17 + 8 = 25.

Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?

In [1]:
# import packages
import pandas as pd
import numpy as np
import operator

In [2]:
directions = pd.read_csv('input_data/Day12.txt', header=None)

directions.columns = ['direction']

directions = directions['direction'].to_list()

In [3]:
def parse_direction(direction):
    '''takes a direction and returns
    an action and a value'''
    
    return direction[0], int(direction[1:])

In [28]:
# Define some global dictionaries
direction_to_degrees = {'E':0, 'N': 90, 'W': 180, 'S': 270}
degrees_to_direction = {0:'E', 90: 'N', 180:'W', 270:'S'}
add_or_sub = {
        'N': operator.add,
        'S': operator.sub,
        'E':operator.add,
        'W':operator.sub,
        'L': operator.add,
        'R': operator.sub }

In [40]:
def process_direction(x, y, orientation, direction):
    '''takes a starting position and direction
    and returns new position and orientation'''
    
    action, value = parse_direction(direction)

    if action in ['N','S']:
        # increase or decrease y position
        y = add_or_sub[action](y, value)
 
    elif action in ['E','W']:
        # increase or decrease x position
        x = add_or_sub[action](x, value)

    elif action =='F':
        if orientation in ['N','S']:
            # increase or decrease y position
            y = add_or_sub[orientation](y, value)
 
        elif orientation in ['E','W']:
            # increase or decrease x position
            x = add_or_sub[orientation](x, value)
        
    else: #action is L or R
        
        new_degrees = add_or_sub[action](direction_to_degrees[orientation], value ) % 360                                       
        orientation = degrees_to_direction[new_degrees]
        
    return x, y, orientation

In [41]:
process_direction(0,0,'E','F10')

(10, 0, 'E')

In [42]:
xpos, ypos = 0,0
orient = 'E'

for direction in directions:
    
#    print(direction, xpos, ypos, orient)
    xpos, ypos, orient = process_direction(xpos, ypos, orient, direction)

In [43]:
xpos, ypos

(-743, 576)

In [44]:
abs(xpos)+abs(ypos)

1319

### Part 2

Before you can give the destination to the captain, you realize that the actual action meanings were printed on the back of the instructions the whole time.

Almost all of the actions indicate how to move a waypoint which is relative to the ship's position:

- Action N means to move the waypoint north by the given value.
- Action S means to move the waypoint south by the given value.
- Action E means to move the waypoint east by the given value.
- Action W means to move the waypoint west by the given value.
- Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
- Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
- Action F means to move forward to the waypoint a number of times equal to the given value.

The waypoint starts 10 units east and 1 unit north relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

For example, using the same instructions as above:

- F10 moves the ship to the waypoint 10 times (a total of 100 units east and 10 units north), leaving the ship at east 100, north 10. The waypoint stays 10 units east and 1 unit north of the ship.
- N3 moves the waypoint 3 units north to 10 units east and 4 units north of the ship. The ship remains at east 100, north 10.
- F7 moves the ship to the waypoint 7 times (a total of 70 units east and 28 units north), leaving the ship at east 170, north 38. The waypoint stays 10 units east and 4 units north of the ship.
- R90 rotates the waypoint around the ship clockwise 90 degrees, moving it to 4 units east and 10 units south of the ship. The ship remains at east 170, north 38.
- F11 moves the ship to the waypoint 11 times (a total of 44 units east and 110 units south), leaving the ship at east 214, south 72. The waypoint stays 4 units east and 10 units south of the ship.

After these operations, the ship's Manhattan distance from its starting position is 214 + 72 = 286.

Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?

In [45]:
def rotate_waypoint(x, y, action, value):
    '''takes an x y pos for a waypoint and rotates it
    cw or ccw based on action and value, returning new x and y pos'''
    
    if (action=='R' and value==90) or (action=='L' and value==270):
        return y, -x
    elif (action=='R' and value==270) or (action=='L' and value==90):
        return -y, x
    else: #action=180 value doesn't matter
        return -x, -y

In [46]:
# Let's test this
rotate_waypoint(10,4,'L',90)

(-4, 10)

In [48]:
def process_direction(x, y, wx, wy, orientation, direction):
    '''takes a starting position for ship and waypoint, and a direction
    and returns a new position for ship and waypoint and orientation'''
    
    action, value = parse_direction(direction)

    if action in ['N','S']:
        # increase or decrease waypoint y position
        wy = add_or_sub[action](wy, value)
    elif action in ['E','W']:
        # increase or decrease waypoint x position
        wx = add_or_sub[action](wx, value)             
    elif action in ['L','R']: 
        # we rotate waypoint cw or ccw from its current position
        wx, wy = rotate_waypoint(wx, wy, action, value)
            
    else: # action == 'F'
        # we move the ship
        x,y = x + value*wx, y + value*wy
        
    return x, y, wx, wy, orientation

In [49]:
xpos, ypos = 0,0
wxpos, wypos = 10,1
orient = 'E'

for direction in directions:
    
    print(direction, xpos, ypos, orient)
    xpos, ypos, wxpos, wypos, orient = process_direction(xpos, ypos, wxpos, wypos, orient, direction)

S1 0 0 E
R270 0 0 E
S5 0 0 E
W2 0 0 E
F63 0 0 E
S3 -126 315 E
L90 -126 315 E
W4 -126 315 E
F59 -126 315 E
S1 -480 197 E
F21 -480 197 E
W4 -606 134 E
R90 -606 134 E
W1 -606 134 E
R180 -606 134 E
S2 -606 134 E
W2 -606 134 E
F91 -606 134 E
W5 -424 -958 E
R270 -424 -958 E
F97 -424 -958 E
R90 740 -1249 E
E2 740 -1249 E
R90 740 -1249 E
F6 740 -1249 E
W1 668 -1243 E
R90 668 -1243 E
W3 668 -1243 E
F1 668 -1243 E
S4 666 -1230 E
F1 666 -1230 E
N3 664 -1221 E
F76 664 -1221 E
S4 512 -309 E
W5 512 -309 E
F88 512 -309 E
W4 -104 395 E
N3 -104 395 E
W2 -104 395 E
F48 -104 395 E
N1 -728 923 E
F50 -728 923 E
E3 -1378 1523 E
F18 -1378 1523 E
L90 -1558 1739 E
F30 -1558 1739 E
S3 -1918 1439 E
R90 -1918 1439 E
W1 -1918 1439 E
N5 -1918 1439 E
W1 -1918 1439 E
R90 -1918 1439 E
R90 -1918 1439 E
E1 -1918 1439 E
S2 -1918 1439 E
F73 -1918 1439 E
R90 -750 52 E
S4 -750 52 E
F89 -750 52 E
F34 -2441 -1728 E
L90 -3087 -2408 E
F11 -3087 -2408 E
S3 -2867 -2617 E
F20 -2867 -2617 E
S5 -2467 -3057 E
E3 -2467 -3057 E
R180 -2

In [50]:
xpos, ypos

(-39392, 23042)

In [51]:
abs(xpos)+abs(ypos)

62434